# 01_data_preparation
**Fecha:** 2025-10-13  
**Versión:** v1

**Descripción:**
Limpieza y preparación del dataset de ecommerce<br>

**Objetivo:** <br>
Preparar la base de datos inicial para el análisis, comenzando por cargar los archivos originales, revisar su estructura general y detectar posibles problemas. En esta etapa se aplican las primeras limpiezas necesarias, como manejo de valores faltantes, tipos de datos y consistencia básica, dejando los datasets listos para los pasos posteriores.

También se documentan las decisiones tomadas durante la limpieza para mantener un registro claro y comprensible de cómo se construyó la versión final de los datos que alimentarán el análisis exploratorio y el resto del proyecto.



In [ ]:
!pip install pandas numpy pyarrow openpyxl

# Imports
import pandas as pd
import numpy as np
from pathlib import Path
import warnings
warnings.filterwarnings('ignore')

print(pd.__version__)

2.2.2


## Carga de datos y primeras observaciones

En esta sección se cargan los archivos originales del proyecto directamente desde el repositorio de GitHub. El propósito es revisar la estructura general de cada dataset y entender con qué tipo de información se trabajará en los siguientes pasos del flujo.

La idea es responder preguntas básicas como:
- ¿Cuántas filas y columnas tiene cada archivo?
- ¿Qué tipos de datos contiene?
- ¿Hay valores nulos o registros duplicados que deban tratarse?
- ¿Las columnas tienen los nombres y formatos esperados?

Estas primeras observaciones permiten identificar posibles ajustes necesarios antes de avanzar hacia la limpieza más puntual.

**Fuente:** Datos sintéticos de un ecommerce en México, inspirados en el esquema de Olist (Kaggle). <br>
**Repositorio:** [https://github.com/RaquelGlez/ecommerce_report](https://github.com/RaquelGlez/ecommerce_report) <br>
Los archivos se encuentran en la carpeta /data/raw/ del repositorio.





In [ ]:
base_url = "https://raw.githubusercontent.com/RaquelGlez/ecommerce_report/refs/heads/main/data/raw/"

# Archivos
orders = pd.read_csv(base_url + "orders.csv", parse_dates=[
    'order_purchase_timestamp', 'order_approved_at', 'order_delivered_carrier_date',
    'order_delivered_customer_date', 'order_estimated_delivery_date'
])
order_items = pd.read_csv(base_url + "order_items.csv", parse_dates=['shipping_limit_date'])
customers = pd.read_csv(base_url + "customers.csv")
payments = pd.read_csv(base_url + "payments.csv")
products = pd.read_csv(base_url + "products.csv")
sellers = pd.read_csv(base_url + "sellers.csv")
reviews = pd.read_csv(base_url + "reviews.csv", parse_dates=[
    'review_creation_date', 'review_answer_timestamp'
])
geo = pd.read_csv(base_url + "geolocation.csv")

print("✅ Archivos cargados correctamente")
print(f"orders: {orders.shape}, order_items: {order_items.shape}, customers: {customers.shape}")


✅ Archivos cargados correctamente
orders: (3000, 8), order_items: (8904, 7), customers: (3000, 5)


### Sobre la carga de datos

Para esta etapa se usó parse_dates en las columnas relacionadas con fechas, de modo que pandas las interpretara correctamente como objetos `datetime64`. Esto facilita los cálculos posteriores relacionados con tiempos, diferencias entre fechas o creación de indicadores. <br>

Los archivos se trajeron directamente desde el repositorio de GitHub utilizando las URLs en formato raw, lo cual permite mantener un flujo de trabajo reproducible sin necesidad de descargar los CSV manualmente.



## Validación inicial de cada dataset

Antes de avanzar hacia cualquier limpieza o proceso de preparación, es importante asegurarnos de que los datos cargados son utilizables y se encuentran en un estado adecuado para continuar con el análisis.

En esta sección se revisan:

- Dimensiones de cada archivo (número de filas y columnas).
- Estructura general, incluyendo nombres de columnas y tipos de datos.
- Presencia de valores nulos o duplicados que puedan requerir atención posterior.
- Observaciones tempranas sobre la calidad y consistencia de la información.

El objetivo de esta validación inicial es tener una primera lectura del estado real de los datasets y anticipar los pasos necesarios en la fase de limpieza.

In [ ]:
def preview_df(df, n=5):
    """Vista rápida de las primeras filas"""
    display(df.head(n))
    print(f"📐 Dimensiones: {df.shape[0]} filas × {df.shape[1]} columnas")


# Vista rápida de cada dataset
dfs = {
    "orders": orders,
    "order_items": order_items,
    "customers": customers,
    "payments": payments,
    "products": products,
    "sellers": sellers,
    "reviews": reviews,
    "geo": geo
}

for name, df in dfs.items():

    print(f"\n📂 Dataset: {name}")
    print("-" * 50)

    preview_df(df, 3)
    print("\nTipos de datos:")
    print(df.dtypes)
    print("-" * 50)





📂 Dataset: orders
--------------------------------------------------


,order_id,customer_id,order_status,order_purchase_timestamp,order_approved_at,order_delivered_carrier_date,order_delivered_customer_date,order_estimated_delivery_date
0,ord_7370c26ead3d3cc6cee4f0548b8d,cus_3b0234601c0276e6cd4e08b6ae67,shipped,2025-08-16 21:12:27.948040,2025-08-16 23:12:27.948040,2025-08-17 23:12:27.948040,NaT,2025-08-24 23:12:27.948040
1,ord_079072dd7970d40a3922a6b0c459,cus_24cb3fa4569c4fd058f954f2e6f9,delivered,2024-12-12 02:52:21.349382,2024-12-12 05:52:21.349382,2024-12-14 05:52:21.349382,2024-12-15 05:52:21.349382,2024-12-15 05:52:21.349382
2,ord_2600b1cb6f6a38610007a8c9142c,cus_67a2c2db5bd7dc2f06a8f9b40226,delivered,2025-06-13 03:04:55.053859,2025-06-13 19:04:55.053859,2025-06-16 19:04:55.053859,2025-06-21 19:04:55.053859,2025-06-21 19:04:55.053859


📐 Dimensiones: 3000 filas × 8 columnas

Tipos de datos:
order_id                                 object
customer_id                              object
order_status                             object
order_purchase_timestamp         datetime64[ns]
order_approved_at                datetime64[ns]
order_delivered_carrier_date     datetime64[ns]
order_delivered_customer_date    datetime64[ns]
order_estimated_delivery_date    datetime64[ns]
dtype: object
--------------------------------------------------

📂 Dataset: order_items
--------------------------------------------------


,order_id,order_item_id,product_id,seller_id,shipping_limit_date,price,freight_value
0,ord_7370c26ead3d3cc6cee4f0548b8d,1,prod_00086,sell_00084,2025-08-18 21:12:27.948040,4469.86,480.12
1,ord_079072dd7970d40a3922a6b0c459,1,prod_00181,sell_00028,2024-12-13 02:52:21.349382,1127.00,149.09
2,ord_2600b1cb6f6a38610007a8c9142c,1,prod_00041,sell_00092,2025-06-16 03:04:55.053859,1977.65,160.03


📐 Dimensiones: 8904 filas × 7 columnas

Tipos de datos:
order_id                       object
order_item_id                   int64
product_id                     object
seller_id                      object
shipping_limit_date    datetime64[ns]
price                         float64
freight_value                 float64
dtype: object
--------------------------------------------------

📂 Dataset: customers
--------------------------------------------------


...

📐 Dimensiones: 100 filas × 4 columnas

Tipos de datos:
seller_id                 object
seller_zip_code_prefix     int64
seller_city               object
seller_state              object
dtype: object
--------------------------------------------------

📂 Dataset: reviews
--------------------------------------------------


,review_id,order_id,review_score,review_comment_title,review_comment_message,review_creation_date,review_answer_timestamp
0,7e849560b5044aa55cbb,ord_7370c26ead3d3cc6cee4f0548b8d,3,Plazo había todavía.,Organización ex después cambio.,2025-08-18 21:12:27.948040,2025-08-19 21:12:27.948040
1,bb6823a894408af2c751,ord_079072dd7970d40a3922a6b0c459,4,Del antonio tratamiento.,Formación particular bueno partir toda. Dice c...,2024-12-18 05:52:21.349382,2024-12-19 05:52:21.349382
2,b79827f880d4a30848ec,ord_2600b1cb6f6a38610007a8c9142c,4,NaN,Hora esfuerzo produce radio octubre. Lugar ade...,2025-06-23 19:04:55.053859,2025-06-24 19:04:55.053859


📐 Dimensiones: 3000 filas × 7 columnas

Tipos de datos:
review_id                          object
order_id                           object
review_score                        int64
review_comment_title               object
review_comment_message             object
review_creation_date       datetime64[ns]
review_answer_timestamp    datetime64[ns]
dtype: object
--------------------------------------------------

📂 Dataset: geo
--------------------------------------------------


,geolocation_zip_code_prefix,geolocation_lat,geolocation_lng,geolocation_city,geolocation_state
0,57344,31.047595,-87.720902,San María los bajos,México
1,98313,22.015389,-87.888243,San Fabiola los bajos,Tamaulipas
2,98313,26.933378,-101.376021,San Catalina de la Montaña,Hidalgo


📐 Dimensiones: 6117 filas × 5 columnas

Tipos de datos:
geolocation_zip_code_prefix      int64
geolocation_lat                float64
geolocation_lng                float64
geolocation_city                object
geolocation_state               object
dtype: object
--------------------------------------------------


In [ ]:
for name, df in dfs.items():
    print(f"✅ {name} — filas: {len(df)}, columnas: {df.shape[1]}")

# Checar claves primarias esperadas
print("\n🔎 Validando claves primarias potenciales...")
print("orders unique order_id:", dfs["orders"].order_id.nunique())
print("customers unique customer_id:", dfs["customers"].customer_id.nunique())
print("sellers unique seller_id:", dfs["sellers"].seller_id.nunique())
print("products unique product_id:", dfs["products"].product_id.nunique())

✅ orders — filas: 3000, columnas: 8
✅ order_items — filas: 8904, columnas: 7
✅ customers — filas: 3000, columnas: 5
✅ payments — filas: 3290, columnas: 5
✅ products — filas: 200, columnas: 9
✅ sellers — filas: 100, columnas: 4
✅ reviews — filas: 3000, columnas: 7
✅ geo — filas: 6117, columnas: 5

🔎 Validando claves primarias potenciales...
orders unique order_id: 3000
customers unique customer_id: 3000
sellers unique seller_id: 100
products unique product_id: 200


### Validación inicial de los datasets

Antes de continuar con cualquier limpieza o transformación, es importante verificar que cada dataset se encuentra completo, estructurado correctamente y listo para integrarse en las siguientes etapas del análisis.
En esta sección realizo una revisión general que incluye:

- Vista rápida de las primeras filas para entender el contenido.
- Dimensiones (filas × columnas) de cada tabla.
- Tipos de datos para confirmar que las columnas coinciden con lo esperado.
- Chequeo básico de claves primarias, como order_id, customer_id, product_id y seller_id, que serán esenciales para unir los diferentes archivos más adelante.
- Estas verificaciones no buscan aún corregir nada, sino:
- Confirmar que los datos se cargaron correctamente.
- Validar que la estructura coincide con el esquema esperado del proyecto.
- Identificar posibles inconsistencias que se atenderán en la etapa de limpieza.

Con esta revisión preliminar confirmamos que los archivos se cargaron correctamente, que cada dataset cuenta con las columnas esperadas y que existen claves que permitirán relacionar las tablas en los siguientes pasos. Esta verificación da una base confiable para avanzar con la limpieza y preparación de datos sin sorpresas posteriores.

## Limpieza y preparación

En esta sección se aplican los ajustes necesarios para dejar los datos en un estado adecuado para el análisis. Esto incluye la revisión de valores faltantes, la estandarización de tipos de datos y la corrección de detalles que podrían afectar los resultados posteriores.

A lo largo del proceso se documentan brevemente las decisiones de limpieza más relevantes, con el fin de mantener claridad y trazabilidad sobre cómo se prepararon los datos.


### Revisión general del estado del dataset

Antes de limpiar, es útil obtener una vista rápida del estado actual del archivo. Aquí reviso el tipo de datos, un resumen estadístico y el número de valores faltantes por columna. Esto ayuda a detectar posibles ajustes necesarios antes de avanzar con transformaciones más específicas.

In [ ]:
orders.info()
orders.describe(include='all')
orders.isna().sum()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3000 entries, 0 to 2999
Data columns (total 8 columns):
 #   Column                         Non-Null Count  Dtype         
---  ------                         --------------  -----         
 0   order_id                       3000 non-null   object        
 1   customer_id                    3000 non-null   object        
 2   order_status                   3000 non-null   object        
 3   order_purchase_timestamp       3000 non-null   datetime64[ns]
 4   order_approved_at              3000 non-null   datetime64[ns]
 5   order_delivered_carrier_date   3000 non-null   datetime64[ns]
 6   order_delivered_customer_date  2075 non-null   datetime64[ns]
 7   order_estimated_delivery_date  3000 non-null   datetime64[ns]
dtypes: datetime64[ns](5), object(3)
memory usage: 187.6+ KB


,0
order_id,0
customer_id,0
order_status,0
order_purchase_timestamp,0
order_approved_at,0
order_delivered_carrier_date,0
order_delivered_customer_date,925
order_estimated_delivery_date,0


### Conversión de tipos de datos
Se ajustan las columnas de fechas y variables categóricas para asegurar que el dataset tenga formatos consistentes antes de continuar con el análisis.

In [ ]:
# Fechas
date_cols = [
    "order_purchase_timestamp",
    "order_approved_at",
    "order_delivered_carrier_date",
    "order_delivered_customer_date",
    "order_estimated_delivery_date"
]

for col in date_cols:
    orders[col] = pd.to_datetime(orders[col], errors="coerce")

# Categóricas
orders["order_status"] = orders["order_status"].astype("category")


### Tratamiento de valores faltantes
Para entender qué tan completo está el dataset, reviso la proporción de valores faltantes por columna. Esto ayuda a distinguir entre datos realmente perdidos y valores que tienen sentido que sean nulos (por ejemplo, pedidos no entregados). Esta vista general permitirá decidir qué columnas requieren tratamiento o simplemente interpretación correcta según la situación.

In [ ]:
missing = orders.isna().mean().sort_values(ascending=False)
missing


,0
order_delivered_customer_date,0.308333
order_id,0.000000
order_status,0.000000
customer_id,0.000000
order_purchase_timestamp,0.000000
order_approved_at,0.000000
order_delivered_carrier_date,0.000000
order_estimated_delivery_date,0.000000


In [ ]:
# Opción fill para fechas faltantes
orders["order_delivered_customer_date"] = orders["order_delivered_customer_date"].fillna(pd.NaT)

En la columna order_delivered_customer_date aparecen valores faltantes porque no todos los pedidos han sido entregados al momento del registro de los datos. <br>
Estos valores no representan errores ni información incompleta, sino pedidos que siguen en proceso.
Por esta razón, no se eliminan ni se rellenan con fechas artificiales; se conservan como valores vacíos (NaT) para reflejar de forma fiel el estado real de las órdenes.

### Corrección de valores inconsistentes

Se revisa que la secuencia de fechas de cada pedido tenga coherencia (compra → aprobación → envío → entrega). Cualquier desviación ayuda a identificar registros que podrían necesitar revisión antes de avanzar.

In [ ]:
# Garantizar fechas ordenadas
orders = orders[
    (orders["order_delivered_customer_date"].isna()) |
    (orders["order_delivered_customer_date"] >= orders["order_purchase_timestamp"])
]

Para garantizar coherencia temporal, se valida que las fechas de entrega no sean anteriores a la fecha de compra.
Se conservan:
- pedidos entregados con fechas válidas
- pedidos aún no entregados (sin fecha de entrega)

Solo se eliminan registros inconsistentes, ya que podrían distorsionar análisis posteriores relacionados con tiempos de entrega.

In [ ]:
# Revisar valores únicos y corregir errores tipográficos si aplica
orders["order_status"].value_counts()

,count
order_status,
delivered,2075
shipped,322
processing,244
invoiced,160
canceled,145
unavailable,54


### Verificación de duplicados
Como parte de la revisión general del dataset, también se revisa si existen filas duplicadas. Esto ayuda a confirmar que cada pedido esté representado una sola vez y que no haya registros repetidos que puedan distorsionar los cálculos posteriores.

En este caso, no se encontraron duplicados, lo cual es coherente con el hecho de que los datos son sintéticos y cada fila fue generada para representar un pedido único.

In [ ]:
for name, df in dfs.items():
    n_dupes = df.duplicated().sum()
    if n_dupes > 0:
        print(f"⚠️ {name.upper()} tiene {n_dupes:,} filas duplicadas.")
    else:
        print(f"✅ {name.upper()} sin filas duplicadas.")

✅ ORDERS sin filas duplicadas.
✅ ORDER_ITEMS sin filas duplicadas.
✅ CUSTOMERS sin filas duplicadas.
✅ PAYMENTS sin filas duplicadas.
✅ PRODUCTS sin filas duplicadas.
✅ SELLERS sin filas duplicadas.
✅ REVIEWS sin filas duplicadas.
✅ GEO sin filas duplicadas.


También se normalizan los nombres de columnas a minúsculas y sin espacios, para mantener consistencia y facilitar la manipulación de los datos en etapas posteriores.

In [ ]:
for name, df in dfs.items():
    df.columns = df.columns.str.strip().str.lower()

In [ ]:
orders.info()
orders.isna().sum()
orders.head()


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3000 entries, 0 to 2999
Data columns (total 8 columns):
 #   Column                         Non-Null Count  Dtype         
---  ------                         --------------  -----         
 0   order_id                       3000 non-null   object        
 1   customer_id                    3000 non-null   object        
 2   order_status                   3000 non-null   category      
 3   order_purchase_timestamp       3000 non-null   datetime64[ns]
 4   order_approved_at              3000 non-null   datetime64[ns]
 5   order_delivered_carrier_date   3000 non-null   datetime64[ns]
 6   order_delivered_customer_date  2075 non-null   datetime64[ns]
 7   order_estimated_delivery_date  3000 non-null   datetime64[ns]
dtypes: category(1), datetime64[ns](5), object(2)
memory usage: 167.3+ KB


,order_id,customer_id,order_status,order_purchase_timestamp,order_approved_at,order_delivered_carrier_date,order_delivered_customer_date,order_estimated_delivery_date
0,ord_7370c26ead3d3cc6cee4f0548b8d,cus_3b0234601c0276e6cd4e08b6ae67,shipped,2025-08-16 21:12:27.948040,2025-08-16 23:12:27.948040,2025-08-17 23:12:27.948040,NaT,2025-08-24 23:12:27.948040
1,ord_079072dd7970d40a3922a6b0c459,cus_24cb3fa4569c4fd058f954f2e6f9,delivered,2024-12-12 02:52:21.349382,2024-12-12 05:52:21.349382,2024-12-14 05:52:21.349382,2024-12-15 05:52:21.349382,2024-12-15 05:52:21.349382
2,ord_2600b1cb6f6a38610007a8c9142c,cus_67a2c2db5bd7dc2f06a8f9b40226,delivered,2025-06-13 03:04:55.053859,2025-06-13 19:04:55.053859,2025-06-16 19:04:55.053859,2025-06-21 19:04:55.053859,2025-06-21 19:04:55.053859
3,ord_aaa7c4626ea7ea636e5bba7afebd,cus_2b70a9c550bfd5e065a756e9fe9b,delivered,2025-04-26 01:55:37.919760,2025-04-26 08:55:37.919760,2025-04-27 08:55:37.919760,2025-04-28 08:55:37.919760,2025-04-29 08:55:37.919760
4,ord_0ca0641b834ceb0a46f85637e125,cus_36ba5c7d572c22a6650f11e37f4d,delivered,2025-06-21 18:52:40.013157,2025-06-21 21:52:40.013157,2025-06-22 21:52:40.013157,2025-06-25 21:52:40.013157,2025-06-27 21:52:40.013157


## Notas finales de la etapa de preparación

Con esta etapa concluye la revisión inicial y la limpieza de los datasets utilizados en el proyecto. Durante el proceso se analizaron sus estructuras, tipos de datos y posibles problemas de calidad para dejar listas las versiones que se utilizarán en las siguientes fases del análisis.

Algunas observaciones relevantes:

- El dataset orders concentra la información temporal del proceso completo de compra y entrega, lo que permitirá estudiar tiempos, retrasos y patrones de cumplimiento.
- En order_items se confirmó que un pedido puede incluir varios productos, algo esperado en este tipo de plataformas.
- Tanto reviews como payments contienen más de un registro por order_id, lo que será importante al momento de relacionar estas tablas.
- Los valores faltantes en la fecha de entrega real reflejan pedidos aún no entregados, por lo que se conservaron como parte del estado natural de la operación.
- No se detectaron duplicados en los datasets, lo cual es consistente con su generación y estructura.

En esta fase se realizaron tareas clave como:
- Identificación y manejo adecuado de valores nulos.
- Conversión y estandarización de tipos de datos (especialmente fechas).
- Revisión de claves y estructura de cada tabla.
- Exportación de las versiones limpias a la carpeta de trabajo para su uso posterior.

Con estos pasos, se deja un conjunto de datos confiable y listo para avanzar hacia el enriquecimiento de variables y el análisis exploratorio, donde podrán observarse comportamientos, patrones y tendencias con mayor claridad.


### Finalizar preparación de datasets

Con la limpieza completada, se guardan los datasets procesados en data/processed/. <br>
Estas versiones serán la base para los siguientes análisis.

In [ ]:
import os

os.makedirs("data/processed", exist_ok=True)
for name, df in dfs.items():
    df.to_csv(f"data/processed/{name}_clean.csv", index=False)


---

### 💾 Exportación de datasets limpios (fase de construcción del proyecto)

Durante la fase inicial del proyecto, los datasets limpios fueron exportados a Google Drive para asegurar persistencia entre sesiones de Google Colab.

En la versión final del proyecto, estos archivos ya se encuentran versionados en el repositorio de GitHub y se consumen directamente desde allí en los notebooks posteriores, por lo que este paso no forma parte del pipeline ejecutable.


---


In [ ]:
# Ruta donde están los datasets limpios
data_path = "/content/data/processed"

# Cargar todos los archivos *_clean.csv
dfs_clean = {}
for file in os.listdir(data_path):
    if file.endswith("_clean.csv"):
        name = file.replace("_clean.csv", "")  # obtener el nombre base
        dfs_clean[name] = pd.read_csv(os.path.join(data_path, file))




In [ ]:
dfs_clean['orders']

,order_id,customer_id,order_status,order_purchase_timestamp,order_approved_at,order_delivered_carrier_date,order_delivered_customer_date,order_estimated_delivery_date
0,ord_7370c26ead3d3cc6cee4f0548b8d,cus_3b0234601c0276e6cd4e08b6ae67,shipped,2025-08-16 21:12:27.948040,2025-08-16 23:12:27.948040,2025-08-17 23:12:27.948040,NaN,2025-08-24 23:12:27.948040
1,ord_079072dd7970d40a3922a6b0c459,cus_24cb3fa4569c4fd058f954f2e6f9,delivered,2024-12-12 02:52:21.349382,2024-12-12 05:52:21.349382,2024-12-14 05:52:21.349382,2024-12-15 05:52:21.349382,2024-12-15 05:52:21.349382
2,ord_2600b1cb6f6a38610007a8c9142c,cus_67a2c2db5bd7dc2f06a8f9b40226,delivered,2025-06-13 03:04:55.053859,2025-06-13 19:04:55.053859,2025-06-16 19:04:55.053859,2025-06-21 19:04:55.053859,2025-06-21 19:04:55.053859
3,ord_aaa7c4626ea7ea636e5bba7afebd,cus_2b70a9c550bfd5e065a756e9fe9b,delivered,2025-04-26 01:55:37.919760,2025-04-26 08:55:37.919760,2025-04-27 08:55:37.919760,2025-04-28 08:55:37.919760,2025-04-29 08:55:37.919760
4,ord_0ca0641b834ceb0a46f85637e125,cus_36ba5c7d572c22a6650f11e37f4d,delivered,2025-06-21 18:52:40.013157,2025-06-21 21:52:40.013157,2025-06-22 21:52:40.013157,2025-06-25 21:52:40.013157,2025-06-27 21:52:40.013157
...,...,...,...,...,...,...,...,...
2995,ord_24c2c43647f31690415f2dba19cf,cus_5372c6084ac0234f6f66e4710457,delivered,2025-07-15 04:39:34.575378,2025-07-15 23:39:34.575378,2025-07-17 23:39:34.575378,2025-07-21 23:39:34.575378,2025-07-23 23:39:34.575378
2996,ord_c5a1dbb9d63acadfc359a07193e1,cus_eea2abb7c1979ed32749b4c3c307,delivered,2024-10-10 02:19:41.678984,2024-10-11 00:19:41.678984,2024-10-12 00:19:41.678984,2024-10-15 00:19:41.678984,2024-10-17 00:19:41.678984
2997,ord_5869f838f0f8c52bdce1ad1e7997,cus_058459cd5c35e10f9a9ab432e96a,canceled,2025-01-18 21:12:54.140170,2025-01-19 05:12:54.140170,2025-01-22 05:12:54.140170,NaN,2025-01-24 05:12:54.140170
2998,ord_8c24c07fa05459b90340f2ea44ba,cus_4dbd83d4fb8a6dcf83b0e6ba4b3c,canceled,2025-07-31 02:19:38.561152,2025-07-31 07:19:38.561152,2025-08-02 07:19:38.561152,NaN,2025-08-07 07:19:38.561152


In [ ]:
# 📂 Guardado de datasets limpios en Google Drive (opcional / fase de construcción)

EXPORT_DATA_TO_DRIVE = False


if EXPORT_DATA_TO_DRIVE:
    from google.colab import drive


  # 1️⃣ Montar Google Drive
    drive.mount('/content/drive')

  # 2️⃣ Definir la ruta persistente dentro de Drive
    data_path = '/content/drive/MyDrive/ecommerce_project/data/processed/'

  # 3️⃣ Crear la carpeta si no existe
    os.makedirs(data_path, exist_ok=True)

  # 4️⃣ Guardar los datasets limpios
    for name, df in dfs_clean.items():
      df.to_csv(f"{data_path}{name}_clean.csv", index=False)


    print("✅ Archivos limpios exportados correctamente a Google Drive:")
    for file in os.listdir(data_path):
        print("   •", file)


condition EXPORT_DATA_TO_DRIVE
